[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## The Discrete Fourier Transform

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import math
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Optional, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions

# Plt the DFT

#===========================Fill This===========================#
# 1. Implement a function for Plotting the DFT.
# 2. The function applies **1D DFT** to a vector or rows of a matrix.
# 3. The function supports common customizations: Log Scale, Normalization, DC Removal, etc...
# !! In many contexts the Log Scale used is `dB`. You may use it here.
# !! The function supports both Complex and Real signals.
# !! You may find `np.fft.fft()` / `np.fft.rfft()` useful.
# !! You may find `np.fft.fftfreq()` / `np.fft.rfftfreq()` useful.
# !! You may find `np.atleast_2d()` useful.
# !! You may find `np.iscomplexobj()` useful.
# !! You may find `np.log1p()` useful.

def PlotDft(mX: np.ndarray,                     #<! Input data (Vector / Matrix)
            samplingFreq: float,                #<! Frequency [Hz]
            /, *, 
            applyDft: bool = True,              #<! Whether the DFt should be applied 
            numFreqBins: Optional[int] = None,  #<! The number of frequency bins to evaluate the DFT on
            singleSide: bool = True,            #<! Show single side (Real signal only)
            logScale: bool = True,              #<! Apply Log Scale transformation
            normalizeData: bool = False,        #<! Normalize signal so peak have value of 1
            removeDc: bool = False,             #<! Zero the DC of the signal
            plotTitle: str = 'DFT',             #<! Title of the plot
            hA: Optional[plt.Axes] = None,      #<! Pre defined axes
            ) -> plt.Axes:

    numDims = np.ndim(mX)

    # Verify data is either a vector
    ???
    
    # Verify data is at least 2D
    mXX = ???
    
    # Check if input is complex
    # If data is complex set (Override) `singleSide` to `False`
    if np.iscomplexobj(???):
        ???
    
    # Define the functions to apply DFT and calculate teh frequency grid
    if singleSide:
        fftFun     = ???
        fftFreqFun = ???
    else:
        fftFun     = ???
        fftFreqFun = ???
    
    # Set the number of bins, handle teh case `numFreqBins` is `None`
    numFreqBins = ???
    
    # Calculate the frequencies grid
    vFftFreq = ???

    # Apply the 1D DFT
    if applyDft:
        mK = ???
    else:
        mK = ???
    
    # Zero the DC component
    if removeDc:
        mK -= ???
    
    # Normalize the data, the peak will have value of 1
    if normalizeData:
        mK /= ???
    
    yLabel = 'Amplitude'
    
    # Apply the Log Scale transformation
    if logScale:
        mK = ??? #<! Safe Log transform (Handles zeros)
        yLabel = 'Amplitude [Log Scale]'


    if hA is None:
        hF, hA = plt.subplots(figsize = (10, 5))

    for ii, vK in enumerate(mK):
        hA.plot(vFftFreq, vK, linewidth = 2, label = f'Signal {(ii + 1):03d}')
    
    hA.set_title(plotTitle)
    hA.set_xlabel('Frequency [Hz]')
    hA.set_ylabel(yLabel)
    hA.legend()

    return hA
#===============================================================#


## The Discrete Fourier Transform (DFT)

The DFT, in the context of Linear Algebra, is basically a [change of basis](https://en.wikipedia.org/wiki/Change_of_basis).  
The DFT basis is composed by Harmonic signals.  
The basis allows analysis of data using an important concept, the Frequency.  

![](https://i.imgur.com/Qqlbz3R.png)
<!-- ![](https://i.postimg.cc/0QRDg1Hc/image.png) -->

The DFT transform is given by:

 - Analysis: $X \left[ k \right] = \sum_{n = 0}^{N - 1} x \left[ n \right] {e}^{-2 \pi j \frac{n k}{N}}$.
 - Synthesis: $x \left[ n \right] = \frac{1}{N} \sum_{k = 0}^{N - 1} X \left[ k \right] {e}^{2 \pi j \frac{n k}{N}}$.

The DFT allows analyzing the spectrum of a signal:

1. The energy of a specific harmonic component.
2. The _Bandwidth_ of the signal (The support).

* <font color='brown'>(**#**)</font> The DFt can be described using a _Unitary_ DFT Matrix $\boldsymbol{D} \in \mathbb{C}^{N \times N}$, given by ${D}_{k, n} = \frac{1}{\sqrt{N}} {e}^{-j 2 \pi \frac{n k}{M}}$.
* <font color='brown'>(**#**)</font> The DFT can be derived as a Uniform Sampling of the [DTFT](https://en.wikipedia.org/wiki/Discrete-time_Fourier_transform).
* <font color='brown'>(**#**)</font> The [Nyquist Shannon Sampling Theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) is closely related to the Fourier Transform.

In [ ]:
# Parameters

numSamples   = 50
samplingFreq = 1 #<! [Hz]
sineFreq     = 0.20

## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

timeInterval = numSamples / samplingFreq
vT = np.linspace(0, timeInterval, numSamples + 1)[:-1] 
vX = np.sin(2 * np.pi * sineFreq * vT)
vN = np.random.randn(numSamples)

## Spectral Leakage

[Spectral Leakage](https://en.wikipedia.org/wiki/Spectral_leakage) is the reason the DFT does not match the intuitive results of Fourier Transform.  
The DFT can be viewed as the sampled DTFT (See [Wikipedia - Sampling the DTFT](https://en.wikipedia.org/wiki/Discrete-time_Fourier_transform#Sampling_the_DTFT)) which in turn can be thought as the result of a sampled windowed signal.


In [ ]:
# Display the Data

# Plot the DFT using `PlotDft()`
hA = PlotDft(vX, samplingFreq, logScale = False, numFreqBins = 10_000);
hA.set_xlim((0.05, 0.35));

* <font color='blue'>(**!**)</font> Change the value of `sineFreq` to random values around `0.2 [Hz]`.
* <font color='red'>(**?**)</font> Why the results are so different?

### The Leakage

From the DTFT point of view, the transform of the Harmonic Signal is the _convolution_ of the Delta Function, by the harmonic signals, by the transform of the _Window Function_ (Rectangle by default).  
In order to have "Delta" like result in the DFT grid should match the zeros of the Window function.  

The "Resolution" of the DFT is: ${F}_{k} = k \frac{{F}_{s}}{N}$.  
Defining ${f}_{c} = \frac{ {F}_{c} }{ {F}_{s} }$ where ${F}_{c}$ is the harmonic signal tone and ${F}_{s}$ is the sampling frequency.  
For a _Rectangle Windows_, Whenever there is an integer $k$ such that ${f}_{c} = k \frac{1}{N}$ then no _Spectral Leakage_ is visible.

![](https://i.imgur.com/ZcnevW7.png)
<!-- ![](https://i.postimg.cc/L67z7xsy/HF7N2hr.png) -->

![](https://i.imgur.com/FD8kC8x.png)
<!-- ![](https://i.postimg.cc/W47q282c/PueR0x6.png) -->


* <font color='brown'>(**#**)</font> See [DSP Illustrations - Spectral Leakage and Zero Padding of the Discrete Fourier Transform](https://dspillustrations.com/pages/posts/misc/spectral-leakage-zero-padding-and-frequency-resolution.html).
* <font color='brown'>(**#**)</font> See [Digital Signals Theory - Spectral Leakage and Windowing](https://brianmcfee.net/dstbook-site/content/ch06-dft-properties/Leakage.html).
* <font color='brown'>(**#**)</font> [Geo's Notepad - Understanding the DFT: Spectral Leakage, Windowing, and Periodicity](https://geo-ant.github.io/blog/2021/dft-spectral-leakage-and-windowing).

## The Power Spectral Density (PSD)

The PSD is the tool to analyze (Wide Sense) Stationary Signals in the Frequency Domain.  
It is defined / calculated in the following theoretically equivalent forms:

 * The DFT of the Auto Correlation Function.
 * The Squared Magnitude of the DFT of the data.

</br>

* <font color='brown'>(**#**)</font> In depth analysis of methods to implement the Gaussian Blur is given in [Pascal Getreuer - A Survey of Gaussian Convolution Algorithms](https://www.ipol.im/pub/art/2013/87).
* <font color='brown'>(**#**)</font> The _Gaussian Kernel_ is used for generating other kernels as well: Derivative Kernels, Laplacian Kernels, etc...
* <font color='brown'>(**#**)</font> For Box Filter like implementation see [Efficient and Accurate Gaussian Image Filtering Using Running Sums](https://ieeexplore.ieee.org/document/6416657).

In [ ]:
# Calculating the Auto Correlation of the Noise

vR = np.correlate(vN, vN, mode = 'full')
vNorm = np.correlate(np.ones(numSamples), np.ones(numSamples), mode = 'full') #<! Counting number of elements per output
vR /= vNorm

* <font color='brown'>(**#**)</font> Read on the different modes of [`np.correlate()`](https://numpy.org/doc/stable/reference/generated/numpy.correlate.html). 

In [ ]:
# Display the Auto Correlation Function

hF, hA = plt.subplots(figsize = (10, 5))

hA.plot(range(-numSamples + 1, numSamples), vR, linewidth = 2)
hA.set_xlabel('Leg (τ)')
hA.set_ylabel('Value')
hA.set_title('The Auto Correlation Function');

* <font color='red'>(**?**)</font> How should the ideal ACF should like for White Noise?

In [ ]:
# Estimation of the PSD

vF, vPsd = sp.signal.welch(vN, fs = samplingFreq, nperseg = 20)
numFreqSamples = len(vF)

In [ ]:
# Display Results

hA = PlotDft(vR, samplingFreq, logScale = False)
hA.plot(vF, vPsd, linewidth = 2, label = 'Welch')
hA.plot(np.fft.rfftfreq(len(vN)), np.square(np.abs(np.fft.rfft(vN))) / numSamples, linewidth = 2, label = 'Square of Magnitude')


hA.legend();

* <font color='red'>(**?**)</font> Explain the different results.

## 2D DFT

In [ ]:
# Load Image
# mI = ski.io.imread('https://i.imgur.com/oErxpp1.png')
mI = ski.io.imread('https://i.postimg.cc/sxH0b5Bp/image.png')
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2)

In [ ]:
# Display Image

hF, hA = plt.subplots(figsize = (10, 5))
hA.imshow(mI, cmap = 'gray', vmin = 0, vmax = 1);

In [ ]:
# Modulate Image

numRows = np.size(mI, 0)
numCols = np.size(mI, 1)

# Aliasing
# signalFreq = math.floor(numRows / 1.25)
# vModulationSignal = 0.15 * np.cos(2 * np.pi * (signalFreq / numRows) * np.arange(numRows))

signalFreq = 0.08
vModulationSignal = 0.15 * np.cos(2 * np.pi * signalFreq * np.arange(numRows))

mM = np.clip(mI + vModulationSignal[:, None], a_min = 0.0, a_max = 1.0)


In [ ]:
# Display Modulated Image
hF, hA = plt.subplots(figsize = (10, 5))
hA.imshow(mM, cmap = 'gray', vmin = 0, vmax = 1);

In [ ]:
# Display Spectrum

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 5))
vHa = vHa.flat

hA = vHa[0]
hA.imshow(np.fft.fftshift(np.log1p(np.abs(np.fft.fft2(mI)))), cmap = 'gray', vmin = 0.0)
hA.set_title('The DFT of the Image')

hA = vHa[1]
hA.imshow(np.fft.fftshift(np.log1p(np.abs(np.fft.fft2(mM)))), cmap = 'gray', vmin = 0.0)
hA.set_title('The DFT of the Modulated Image')